In [22]:
from langchain_core.messages import ChatMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.embeddings import Embeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_community.document_loaders import TextLoader, PyPDFLoader, PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

from langchain_openai import ChatOpenAI

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langserve.pydantic_v1 import BaseModel, Field
from typing import List, Union

from openai import OpenAI
from typing import List

import os
import warnings
warnings.filterwarnings("ignore")

In [23]:
# PyMuPDFLoader 을 이용해 PDF 파일 로드
loader = PyMuPDFLoader("file/constitution.pdf")
pages = loader.load()

In [24]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=20,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""], 
    length_function=len,
)
docs = text_splitter.split_documents(pages)

In [25]:
from openai import OpenAI

class MyEmbeddings(Embeddings):
    def __init__(self, base_url, api_key="lm-studio"):
        self.client = OpenAI(base_url=base_url, api_key=api_key)

    def embed_documents(self, texts: List[str], model="nomic-ai/nomic-embed-text-v1.5-GGUF") -> List[List[float]]:
        texts = list(map(lambda text:text.replace("\n", " "), texts))
        datas = self.client.embeddings.create(input=texts, model=model).data
        return list(map(lambda data:data.embedding, datas))
        
    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

RAG_PROMPT_TEMPLATE = "You always answer into Korean. You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use all the sentences and keep the answer concise in bullet.\nQuestion: {question} \nContext: {context} \nAnswer:"

SYS_PROMPT_TEMPLATE = "당신은 ** 에이전트입니다. 가능한 한 도움이 될 수 있도록 사실적인 정보만을 제공하도록 설계되었습니다. 친절하게 응대하지만 지나치게 수다스럽지는 않습니다. 아래에 제공된 컨텍스트 정보를 바탕으로, 사전 지식 없이 질문에 답하십시오."

In [40]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 언어 모델
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
    temperature=0.5,
)

# 임베딩 모델
emb = MyEmbeddings(base_url="http://localhost:1234/v1")

# RAG 체인
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "You always answer into Korean."),
    ("user", RAG_PROMPT_TEMPLATE),
])
rag_chain = rag_prompt | llm | StrOutputParser()

In [27]:
vectorstore = FAISS.from_documents(docs, embedding=emb, distance_strategy=DistanceStrategy.COSINE)
retriever = vectorstore.as_retriever(search_kwargs={'k':3})

In [41]:
format_docs = lambda docs:"\n\n".join(doc.page_content for doc in docs)
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_chain
)

In [46]:
chain_input = "내가 직업을 강요받았어. 이건 헌법 어디에 위배 되는 거야?"

# 응답을 한 번에 받아오기
msg = chain.invoke(chain_input)

# 결과 출력
print(msg)

1. 직업 강요는 개인의 자유와 권리를 침해하는 것으로, 헌법에 명시된 인권 및 기본권리 규정에 위배됩니다.
2. 특히 제32조 ①항은 모든 국민이 근로의 의무를 진다고 규정하고 있으며, 국가는 법률로 그 내용을 정해야 한다고 명시하고 있습니다. 이 조항을 통해 강제 노동이나 직업 강요를 금지하고 있음을 알 수 있습니다.
3. 또한, 제10조에 따르면 모든 국민은 인간으로서의 존엄과 가치를 가지며 행복을 추구할 권리를 가집니다. 직업 강요는 이러한 권리 침해로 볼 수 있으며 헌법 위반입니다.
4. 마지막으로, 제37조 ②항은 법률이 국민의 자유와 권리의 본질적인 내용을 침해해서는 안 된다고 명시하고 있습니다. 직업 강요 또한 개인의 자유와 권리를 침해하는 것으로 이 조항에도 위배됩니다.
